In [19]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [20]:
from tensorflow.keras.models import Model,Sequential
from tensorflow.keras.layers import Dense,Dropout

In [21]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [22]:
path = '/content/ecg_autoencoder_dataset.csv'

In [23]:
df = pd.read_csv(path,header=None)
df

,0,1,2,3,4,5,6,7,8,9,...,131,132,133,134,135,136,137,138,139,140
0,-0.112522,-2.827204,-3.773897,-4.349751,-4.376041,-3.474986,-2.181408,-1.818286,-1.250522,-0.477492,...,0.792168,0.933541,0.796958,0.578621,0.257740,0.228077,0.123431,0.925286,0.193137,1
1,-1.100878,-3.996840,-4.285843,-4.506579,-4.022377,-3.234368,-1.566126,-0.992258,-0.754680,0.042321,...,0.538356,0.656881,0.787490,0.724046,0.555784,0.476333,0.773820,1.119621,-1.436250,1
2,-0.567088,-2.593450,-3.874230,-4.584095,-4.187449,-3.151462,-1.742940,-1.490659,-1.183580,-0.394229,...,0.886073,0.531452,0.311377,-0.021919,-0.713683,-0.532197,0.321097,0.904227,-0.421797,1
3,0.490473,-1.914407,-3.616364,-4.318823,-4.268016,-3.881110,-2.993280,-1.671131,-1.333884,-0.965629,...,0.350816,0.499111,0.600345,0.842069,0.952074,0.990133,1.086798,1.403011,-0.383564,1
4,0.800232,-0.874252,-2.384761,-3.973292,-4.338224,-3.802422,-2.534510,-1.783423,-1.594450,-0.753199,...,1.148884,0.958434,1.059025,1.371682,1.277392,0.960304,0.971020,1.614392,1.421456,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4993,0.608558,-0.335651,-0.990948,-1.784153,-2.626145,-2.957065,-2.931897,-2.664816,-2.090137,-1.461841,...,1.757705,2.291923,2.704595,2.451519,2.017396,1.704358,1.688542,1.629593,1.342651,0
4994,-2.060402,-2.860116,-3.405074,-3.748719,-3.513561,-3.006545,-2.234850,-1.593270,-1.075279,-0.976047,...,1.388947,2.079675,2.433375,2.159484,1.819747,1.534767,1.696818,1.483832,1.047612,0
4995,-1.122969,-2.252925,-2.867628,-3.358605,-3.167849,-2.638360,-1.664162,-0.935655,-0.866953,-0.645363,...,-0.472419,-1.310147,-2.029521,-3.221294,-4.176790,-4.009720,-2.874136,-2.008369,-1.808334,0
4996,-0.547705,-1.889545,-2.839779,-3.457912,-3.929149,-3.966026,-3.492560,-2.695270,-1.849691,-1.374321,...,1.258419,1.907530,2.280888,1.895242,1.437702,1.193433,1.261335,1.150449,0.804932,0


In [38]:
df.dropna(inplace=True)

In [39]:
features = df.drop(140,axis=1)
targets = df[140]

In [40]:
features.shape , targets.shape

((4998, 140), (4998,))

In [41]:
x_train,x_test,y_train,y_test = train_test_split(features,targets,test_size=0.2)

In [42]:
train_index = y_train[y_train == 1].index
train_data = x_train.loc[train_index]

In [44]:
min_max_scaler = MinMaxScaler(feature_range = (0,1))
train_data_scaled = min_max_scaler.fit_transform(train_data.copy())
test_data_scaled = min_max_scaler.transform(x_test)

In [51]:
class AutoEncoder(Model):
  def __init__(self,output_dim,ldim=8):
    super().__init__()
    self.encoder = Sequential([
        Dense(64,activation='relu'),
        Dropout(0.2),
        Dense(32,activation='relu'),
        Dropout(0.2),
        Dense(16,activation='relu'),
        Dropout(0.2),
        Dense(ldim,activation='relu'),
        Dropout(0.2),
    ])
    self.decoder = Sequential([
        Dense(16,activation='relu'),
        Dropout(0.2),
        Dense(32,activation='relu'),
        Dropout(0.2),
        Dense(64,activation='relu'),
        Dropout(0.2),
        Dense(output_dim,activation='sigmoid'),
        Dropout(0.2),
    ])
  def call(self,input):
    encoded = self.encoder(input)
    decoded = self.decoder(encoded)
    return decoded

In [52]:
model = AutoEncoder(output_dim = test_data_scaled.shape[1])

In [53]:
from tensorflow.keras.losses import MeanSquaredLogarithmicError

In [54]:
model.compile(optimizer='adam',loss='msle',metrics=['mse'])

In [57]:
H = model.fit(train_data_scaled,train_data_scaled,epochs=10,validation_data=(test_data_scaled,test_data_scaled))

Epoch 1/10
73/73 ━━━━━━━━━━━━━━━━━━━━ 5s 12ms/step - loss: 0.0084 - mse: 0.0189 - val_loss: 0.0087 - val_mse: 0.0203
Epoch 2/10
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0036 - mse: 0.0080 - val_loss: 0.0078 - val_mse: 0.0181
Epoch 3/10
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0025 - mse: 0.0056 - val_loss: 0.0078 - val_mse: 0.0180
Epoch 4/10
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0024 - mse: 0.0054 - val_loss: 0.0075 - val_mse: 0.0174
Epoch 5/10
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0020 - mse: 0.0046 - val_loss: 0.0072 - val_mse: 0.0167
Epoch 6/10
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0019 - mse: 0.0041 - val_loss: 0.0066 - val_mse: 0.0153
Epoch 7/10
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0017 - mse: 0.0039 - val_loss: 0.0062 - val_mse: 0.0145
Epoch 8/10
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0016 - mse: 0.0036 - val_loss: 0.0061 - val_mse: 0.0142
Epoch 9/10
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0015 - mse:

In [67]:
def get_theshold(model,train_data_scaled):
  reconstuctor = model.predict(train_data_scaled)
  reconstructor_error = tf.keras.losses.msle(reconstuctor,train_data_scaled)
  threshold = np.mean(reconstructor_error.numpy()) + np.std(reconstructor_error.numpy())
  return threshold

def get_predictions(model,threshold,test_data_scaled):
  predictions = model.predict(test_data_scaled)
  error = tf.keras.losses.msle(predictions,test_data_scaled).numpy()
  anomaly_mask = error>threshold
  preds = np.where(anomaly_mask, 0.0, 1.0)
  return error,pred

In [68]:
threshold = get_theshold(model,train_data_scaled)
threshold

73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


0.0037376886826066148

In [69]:
error,predds = get_predictions(model,threshold,test_data_scaled)

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step


In [70]:
num_anamolies  = np.sum(error>threshold)
num_anamolies

430

In [ ]:
#check graph later